# Imports & Load

In [ ]:
import os
import time
import logging
import requests
import pandas as pd
from pprint import pprint
from geopy.geocoders import Nominatim
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading
from tqdm.notebook import tqdm
#from tqdm import tqdm
# !pip install --upgrade tqdm

In [ ]:
# arquivo_excel = "amostra dados mackenzie.xlsx"
# df = pd.read_excel(arquivo_excel, engine='openpyxl')
# # Remova os espaços no início dos nomes das colunas
# df.columns = [col.strip() for col in df.columns]

df = pd.read_csv("tudo SETEMBRO 2023.csv", sep = ';')

<ipython-input-2-7de51aa263a3>:6: DtypeWarning: Columns (40,46,71,91) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("tudo SETEMBRO 2023.csv", sep = ';')


In [ ]:
# df.to_csv('raw.csv', encoding = 'utf-8', sep = ';', index = True)
# df = pd.read_csv("raw.csv", sep = ';')
# df.to_excel('raw.xlsx', encoding = 'utf-8')

# Explore & Transform

In [ ]:
print(list(df.columns))
print(df.shape)

['d.cd_ibge', ' d.cod_familiar_fam', ' d.dat_cadastramento_fam', ' d.dat_atual_fam', ' d.cod_est_cadastral_fam', ' d.cod_forma_coleta_fam', ' d.dta_entrevista_fam', ' d.nom_localidade_fam', ' d.nom_tip_logradouro_fam', ' d.nom_titulo_logradouro_fam', ' d.nom_logradouro_fam', ' d.num_logradouro_fam', ' d.des_complemento_fam', ' d.des_complemento_adic_fam', ' d.num_cep_logradouro_fam', ' d.cod_unidade_territorial_fam', ' d.nom_unidade_territorial_fam', ' d.txt_referencia_local_fam', ' d.nom_entrevistador_fam', ' d.num_cpf_entrevistador_fam', ' d.vlr_renda_media_fam', ' d.fx_rfpc', ' d.vlr_renda_total_fam', ' d.marc_pbf', ' d.qtde_meses_desat_cat', ' d.cod_local_domic_fam', ' d.cod_especie_domic_fam', ' d.qtd_comodos_domic_fam', ' d.qtd_comodos_dormitorio_fam', ' d.cod_material_piso_fam', ' d.cod_material_domic_fam', ' d.cod_agua_canalizada_fam', ' d.cod_abaste_agua_domic_fam', ' d.cod_banheiro_domic_fam', ' d.cod_escoa_sanitario_domic_fam', ' d.cod_destino_lixo_domic_fam', ' d.cod_ilumin

In [ ]:
fields = [
    "uf",
    "d.nom_tip_logradouro_fam",
    "d.nom_titulo_logradouro_fam",
    "d.nom_logradouro_fam",
    "d.num_logradouro_fam",
    "d.des_complemento_fam",
    "d.des_complemento_adic_fam",
    "d.num_cep_logradouro_fam"
]

# Primeiro, vamos remover os espaços em branco no início dos nomes das colunas
df.columns = df.columns.str.lstrip()

# Agora podemos tentar filtrar as colunas novamente
# Vamos garantir que a lista 'fields' também não tenha espaços no início.
fields = [field.lstrip() for field in fields]

# Verificamos quais colunas existem no DataFrame após a limpeza
existing_fields = [field for field in fields if field in df.columns]

# Filtrar o DataFrame para manter apenas as colunas que existem
df = df[existing_fields]

# Determinar constantes da cidade do arquivo (se mudar arquivo, comentar)
df['uf'] = 'MA'
df['munic'] = 'São Luís'

# To numeric
df['d.num_logradouro_fam'] = pd.to_numeric(df['d.num_logradouro_fam'], errors='coerce').astype('Int64')

df.columns

<ipython-input-5-7f4c6d586ed0>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['uf'] = 'MA'
<ipython-input-5-7f4c6d586ed0>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['munic'] = 'São Luís'
<ipython-input-5-7f4c6d586ed0>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

Index(['d.nom_tip_logradouro_fam', 'd.nom_titulo_logradouro_fam',
       'd.nom_logradouro_fam', 'd.num_logradouro_fam', 'd.des_complemento_fam',
       'd.des_complemento_adic_fam', 'd.num_cep_logradouro_fam', 'uf',
       'munic'],
      dtype='object')

In [ ]:
df.head()

,d.nom_tip_logradouro_fam,d.nom_titulo_logradouro_fam,d.nom_logradouro_fam,d.num_logradouro_fam,d.des_complemento_fam,d.des_complemento_adic_fam,d.num_cep_logradouro_fam,uf,munic
0,RUA,NaN,QUENIA,2,B,QD 40,65080000,MA,São Luís
1,RUA,NaN,QUENIA,2,B,QD 40,65080000,MA,São Luís
2,RUA,NaN,QUENIA,2,B,QD 40,65080000,MA,São Luís
3,RUA,NaN,QUENIA,2,B,QD 40,65080000,MA,São Luís
4,RUA,NaN,SAO FRANCISCO,25,NaN,NaN,65092009,MA,São Luís


In [ ]:
# Função para concatenar as colunas e formar um endereço
def formatar_endereco(linha):
    partes_endereco = {
        'tipo_logradouro': str(linha['d.nom_tip_logradouro_fam']).strip() if pd.notna(linha['d.nom_tip_logradouro_fam']) else '',
        'titulo_logradouro': str(linha['d.nom_titulo_logradouro_fam']).strip() if pd.notna(linha['d.nom_titulo_logradouro_fam']) else '',
        'nome_logradouro': str(linha['d.nom_logradouro_fam']).strip() if pd.notna(linha['d.nom_logradouro_fam']) else '',
        'numero': str(linha['d.num_logradouro_fam']).strip() if pd.notna(linha['d.num_logradouro_fam']) else '',
        'cep': str(linha['d.num_cep_logradouro_fam']).strip() if pd.notna(linha['d.num_cep_logradouro_fam']) else '',
        'municipio': str(linha['munic']).strip() if pd.notna(linha['munic']) else '',
        'uf': str(linha['uf']).strip() if pd.notna(linha['uf']) else ''
    }

    # Formatar o CEP
    cep = str(linha['d.num_cep_logradouro_fam']).strip()
    # Remover o sufixo '.0' se ele existir, resultante da conversão de float para string
    cep = cep.split('.')[0]  # Isso garante que qualquer parte decimal será removida
    if len(cep) == 8:
        cep = f"{cep[:5]}-{cep[5:]}"
    partes_endereco['cep'] = cep  # Agora, atualizamos partes_endereco com o CEP formatado


    endereco_formatado = f"{partes_endereco['tipo_logradouro']} {partes_endereco['titulo_logradouro']} {partes_endereco['nome_logradouro']}"

    if partes_endereco['numero']:
        endereco_formatado += f", {partes_endereco['numero']}"
    if partes_endereco['municipio']:
        endereco_formatado += f", {partes_endereco['municipio']}"
    if partes_endereco['uf']:
        endereco_formatado += f" - {partes_endereco['uf']}"
    if partes_endereco['cep']:
        endereco_formatado += f", {partes_endereco['cep']}"

    return endereco_formatado.strip()

# Aplique a função a cada linha do DataFrame
df['endereco_completo'] = df.apply(formatar_endereco, axis=1)

In [ ]:
df['endereco_completo'][0]

KeyError: ignored

# Coletar dados via API Bing

In [ ]:
# Descomente essa célula para testar em uma ou poucas linhas
#df = df[0:10]

# # Sua chave de API do Bing Maps
# api_key = 'AsswqDO1ozUlMyOVO9idR24SFbgLPHjq5pPcWAv1YYWQF2HAw61RHtmyF3h69tEf'

# # O endereço que você deseja geocodificar
# endereco = 'Rua Doutora Romilda Valentino, 140. São Paulo - SP.'

# # Construa a URL da API
# url = f'http://dev.virtualearth.net/REST/v1/Locations?query={endereco}&key={api_key}'

# # Faça a requisição à API
# response = requests.get(url)

# # Verifique se a requisição foi bem sucedida
# if response.status_code == 200:
#     data = response.json()
#     pprint(data)
# else:
#     print('Falha na requisição:', response.status_code)

In [ ]:
# Dê load no arquivo em cache, para voltar de onde parou (se é a primeira vez rodando, comente essa linha e siga o restante)
df = pd.read_csv('geocoding_progress.csv', sep=';')

# Configurando o logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Sua chave de API do Bing Maps

# Eu
# api_key = 'AsswqDO1ozUlMyOVO9idR24SFbgLPHjq5pPcWAv1YYWQF2HAw61RHtmyF3h69tEf'

# coolrunnings1312@hotmail.com
# Habyarimana
# api_key = 'ArxZIDAqIhBIgMekV0TVf4Mmsl8fFzYHrgy7fnFSmg7tQBkS581cBW3-DRqMPE8l'


# coolrunnings1213@hotmail.com
# RecepTayyip
# api_key = 'AlieXZLF0cU6w2u0OjbdyaLSj7ytu-m3Oc0YbLMgOuhMsY35m6Uruv96-EDvOxwm'


# coolrunnings0810@hotmail.com
# MustafaKemal
api_key = 'AnmY9Xh_RTE6AlwBzCKshDvS-n4IzDlLhlE0pcXT55YSQWx5WyiV8U7QEPIa4e0P'


# Função para geocodificar um endereço com tentativas de nova execução usando Bing Maps
def geocodificar_bing(endereco, api_key):
    tentativas = 0
    max_tentativas = 5
    espera = 1  # espera de 1 segundo antes de tentar novamente

    while tentativas < max_tentativas:
        try:
            # Construa a URL da API
            url = f'http://dev.virtualearth.net/REST/v1/Locations?query={endereco}&key={api_key}'
            # Faça a requisição à API
            response = requests.get(url, timeout=10)

            if response.status_code == 200:
                data = response.json()
                return data
            else:
                logger.warning("Falha na requisição para o endereço %s: %s", endereco, response.status_code)
                tentativas += 1
                time.sleep(espera)
                espera *= 2  # dobra o tempo de espera a cada tentativa
        except requests.exceptions.RequestException as e:
            logger.warning("Erro na requisição para o endereço %s: %s. Tentativa %d de %d",
                            endereco, str(e), tentativas + 1, max_tentativas)
            tentativas += 1
            time.sleep(espera)
            espera *= 2  # dobra o tempo de espera a cada tentativa

    logger.error("Número máximo de tentativas atingido para o endereço: %s", endereco)
    return None

# Inicializa a coluna 'localizacao' com NaN se ela não existir
if 'localizacao' not in df.columns:
    df['localizacao'] = pd.NA

# Variável para controlar quando salvar os dados
save_interval = 4000  # Define o intervalo de salvamento

# Prepare um lock para sincronizar o acesso ao DataFrame e ao arquivo CSV
lock = threading.Lock()

def save_df(df, filename):
    with lock:
        df.to_csv(filename, index=False)

# Use ThreadPoolExecutor para paralelizar a execução da função de geocodificação
num_cpus = os.cpu_count() - 1
with ThreadPoolExecutor(max_workers=num_cpus) as executor:
    # Prepare um dicionário para manter os futuros
    futures = {}
    for index, row in df.iterrows():
        if pd.isna(row['localizacao']):  # Verifica se a 'localizacao' ainda não foi definida
            futures[executor.submit(geocodificar_bing, row['endereco_completo'], api_key)] = index

    # Itera através dos futuros e salva os resultados
    for i, future in enumerate(tqdm(as_completed(futures), total=len(futures), desc="Geocodificação")):
        index = futures[future]
        resultado = future.result()
        with lock:
            if resultado:
                df.at[index, 'localizacao'] = resultado  # Atualiza o DataFrame pelo índice
            else:
                df.at[index, 'localizacao'] = None  # Marca como None para falhas

        # Verifica se é hora de salvar os dados
        if i % save_interval == 0 and i > 0:
            save_df(df, 'geocoding_progress.csv')  # Salva os progressos no arquivo CSV

# Salva os resultados finais após a conclusão
save_df(df, 'geocoding_final.csv')

KeyError: ignored

# Validate

In [ ]:
# df = pd.read_csv('geocoding_progress.csv')
# df = pd.read_csv('geocoding_final.csv')

df.head()

,d.nom_tip_logradouro_fam,d.nom_titulo_logradouro_fam,d.nom_logradouro_fam,d.num_logradouro_fam,d.des_complemento_fam,d.des_complemento_adic_fam,d.num_cep_logradouro_fam,uf,munic,endereco_completo,localizacao
0,RUA,NaN,QUENIA,2,B,QD 40,65080000,MA,São Luís,"RUA QUENIA, 2, São Luís - MA, 65080-000",{'authenticationResultCode': 'ValidCredentials...
1,RUA,NaN,QUENIA,2,B,QD 40,65080000,MA,São Luís,"RUA QUENIA, 2, São Luís - MA, 65080-000",{'authenticationResultCode': 'ValidCredentials...
2,RUA,NaN,QUENIA,2,B,QD 40,65080000,MA,São Luís,"RUA QUENIA, 2, São Luís - MA, 65080-000",{'authenticationResultCode': 'ValidCredentials...
3,RUA,NaN,QUENIA,2,B,QD 40,65080000,MA,São Luís,"RUA QUENIA, 2, São Luís - MA, 65080-000",{'authenticationResultCode': 'ValidCredentials...
4,RUA,NaN,SAO FRANCISCO,25,NaN,NaN,65092009,MA,São Luís,"RUA SAO FRANCISCO, 25, São Luís - MA, 65092-009",{'authenticationResultCode': 'ValidCredentials...


In [ ]:
df['localizacao'].head()

0    {'authenticationResultCode': 'ValidCredentials...
1    {'authenticationResultCode': 'ValidCredentials...
2    {'authenticationResultCode': 'ValidCredentials...
3    {'authenticationResultCode': 'ValidCredentials...
4    {'authenticationResultCode': 'ValidCredentials...
Name: localizacao, dtype: object

In [ ]:
df['localizacao'][1]
null_count = df['localizacao'].isna().sum()
null_count

0

In [ ]:
df['localizacao'][1]

{'authenticationResultCode': 'ValidCredentials',
 'brandLogoUri': 'http://dev.virtualearth.net/Branding/logo_powered_by.png',
 'copyright': 'Copyright © 2023 Microsoft and its suppliers. All rights reserved. This API cannot be accessed and the content and any results may not be used, reproduced or transmitted in any manner without express written permission from Microsoft Corporation.',
 'resourceSets': [{'estimatedTotal': 1,
   'resources': [{'__type': 'Location:http://schemas.microsoft.com/search/local/ws/rest/v1',
     'bbox': [-2.5682955175706765,
      -44.3352859375376,
      -2.5605700824293236,
      -44.32497506246241],
     'name': 'Rua Quênia, 2, Anjo da Guarda, São Luís - Maranhão, 65085-102, Brazil',
     'point': {'type': 'Point', 'coordinates': [-2.5644328, -44.3301305]},
     'address': {'addressLine': 'Rua Quênia 2',
      'adminDistrict': 'Maranhão',
      'countryRegion': 'Brazil',
      'formattedAddress': 'Rua Quênia, 2, Anjo da Guarda, São Luís - Maranhão, 65085-1

In [ ]:
import pandas as pd

# Suponha que a coluna 'localizacao' contenha o dicionário com 'coordinates'
df['latitude'] = df['localizacao'].apply(lambda x: x['resourceSets'][0]['resources'][0]['point']['coordinates'][0])
df['longitude'] = df['localizacao'].apply(lambda x: x['resourceSets'][0]['resources'][0]['point']['coordinates'][1])


In [ ]:
df.to_csv('teste_lat_long.csv', sep = ';', encoding = 'utf-8', index = True)
df = pd.read_csv("teste_lat_long.csv", sep = ';')
df.to_excel('teste_lat_long.xlsx', encoding = 'utf-8')

In [ ]:
df_4 = pd.read_csv('teste_lat_long.csv', sep = ';')
df_4.head()

,Unnamed: 0,d.nom_tip_logradouro_fam,d.nom_titulo_logradouro_fam,d.nom_logradouro_fam,d.num_logradouro_fam,d.des_complemento_fam,d.des_complemento_adic_fam,d.num_cep_logradouro_fam,uf,munic,endereco_completo,localizacao,latitude,longitude
0,0,RUA,NaN,QUENIA,2,B,QD 40,65080000,MA,São Luís,"RUA QUENIA, 2, São Luís - MA, 65080-000",{'authenticationResultCode': 'ValidCredentials...,-2.564433,-44.330131
1,1,RUA,NaN,QUENIA,2,B,QD 40,65080000,MA,São Luís,"RUA QUENIA, 2, São Luís - MA, 65080-000",{'authenticationResultCode': 'ValidCredentials...,-2.564433,-44.330131
2,2,RUA,NaN,QUENIA,2,B,QD 40,65080000,MA,São Luís,"RUA QUENIA, 2, São Luís - MA, 65080-000",{'authenticationResultCode': 'ValidCredentials...,-2.564433,-44.330131
3,3,RUA,NaN,QUENIA,2,B,QD 40,65080000,MA,São Luís,"RUA QUENIA, 2, São Luís - MA, 65080-000",{'authenticationResultCode': 'ValidCredentials...,-2.564433,-44.330131
4,4,RUA,NaN,SAO FRANCISCO,25,NaN,NaN,65092009,MA,São Luís,"RUA SAO FRANCISCO, 25, São Luís - MA, 65092-009",{'authenticationResultCode': 'ValidCredentials...,-2.554173,-44.229532


# Outras APIs

In [ ]:
# from geopy.geocoders import Nominatim
# from pprint import pprint

# # Instancie um novo cliente Nominatim
# app = Nominatim(user_agent="ibge_location")

# # Defina a localização diretamente
# your_loc = "Rua Doutora Romilda Valentino, 140. São Paulo - SP."

# # Obtenha a localização bruta a partir do endereço
# location = app.geocode(your_loc)

# # Verifique se a localização foi encontrada e imprima os dados brutos
# if location:
#     pprint(location.raw)
# else:
#     print("A localização não pôde ser encontrada.")
# # Configurando o logging
# logging.basicConfig(level=logging.INFO)
# logger = logging.getLogger(__name__)

# # Instancie um novo cliente Nominatim
# app = Nominatim(user_agent="geocode_ibge")

# # Função para geocodificar um endereço com tentativas de nova execução
# def geocodificar(endereco):
#     tentativas = 0
#     max_tentativas = 5
#     espera = 1  # espera de 1 segundo antes de tentar novamente

#     while tentativas < max_tentativas:
#         try:
#             location = app.geocode(endereco, timeout=10)  # Aumentando o tempo limite para 10 segundos
#             if location:
#                 return location.raw
#             else:
#                 logger.warning("A localização não pôde ser encontrada para o endereço: %s", endereco)
#                 return None
#         except Exception as e:
#             if "ConnectTimeoutError" in str(e) or "NewConnectionError" in str(e):
#                 logger.warning("Timeout na conexão. Aguardando para tentar novamente...")
#                 time.sleep(espera)
#                 tentativas += 1
#                 espera *= 2  # dobra o tempo de espera a cada tentativa
#             else:
#                 logger.error("Erro ao geocodificar o endereço %s: %s", endereco, str(e))
#                 return None
#     logger.error("Número máximo de tentativas atingido para o endereço: %s", endereco)
#     return None

# # Lista para armazenar os resultados
# resultados = []

# # Use ThreadPoolExecutor para paralelizar a execução da função de geocodificação
# num_cpus = os.cpu_count()
# with ThreadPoolExecutor(max_workers=1) as executor:
#     futures = {executor.submit(geocodificar, endereco): endereco for endereco in df_2['endereco_completo']}
#     for future in tqdm(as_completed(futures), total=len(futures), desc="Geocodificação"):
#         endereco = futures[future]
#         try:
#             resultados.append(future.result())
#         except Exception as e:
#             logger.error("Erro ao geocodificar o endereço %s: %s", endereco, str(e))
#             resultados.append(None)

# # Adicione os resultados ao DataFrame
# df_2['localizacao'] = resultados

# tqdm.pandas(desc="Progresso da Geocodificação")

# # Inicialize um novo cliente Nominatim
# app = Nominatim(user_agent="ibge_location")

# # Função para geocodificar um endereço e retornar a localização bruta
# def geocodificar(linha):
#     endereco = ', '.join(linha.dropna().astype(str))
#     location = app.geocode(endereco)
#     if location:
#         return location.raw
#     else:
#         return "Não foi possível geocodificar o endereço"

# # Supondo que df_2 seja o seu DataFrame

# # Aplique a função de geocodificação a cada linha do DataFrame com uma barra de progresso
# df_2['localizacao'] = df_2.progress_apply(geocodificar, axis=1)